###  

# SQL - FINAL PROJECT - Masterschool

The coronavirus took the entire world by surprise, changing everyone's daily routine. City dwellers no longer spent their free time outside, going to cafes and malls; more people were at home, reading books. That attracted the attention of startups that rushed to develop new apps for book lovers. 

You've been given a database of one of the services competing in this market. It contains data on books, publishers, authors, and customer ratings and reviews of books. This information will be used to generate a value proposition for a new product. 

### Task

1. Find the number of books released after January 1, 2000.
2. Find the number of user reviews and the average rating for each book.
3. Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude    brochures and similar publications from your analysis).
4. Identify the author with the highest average book rating (look only at books with at least 50 ratings).
5. Find the average number of text reviews among users who rated more than 50 books.

###  Instructions for completing the task

- Describe the goals of the study.
- Study the tables (print the first rows).
- Make an SQL query for each of the tasks.
- Output the results of each query in the Notebook.
- Describe your conclusions for each of the tasks.

In [13]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'praktikum_student',         # user name
             'pwd': 'Sdf4$2;d-d30pp', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

books= pd.read_sql_table("books",engine)
books.head()

authors=pd.read_sql_table("authors", engine)
authors.head()

ratings= pd.read_sql_table("ratings", engine)
ratings.head()

reviews=pd.read_sql_table("ratings", engine)
reviews.head()

publishers=pd.read_sql_table("publishers", engine)
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### 1. Find the number of books released after January 1, 2000

In [2]:
query1='''
SELECT COUNT(num_pages) 
FROM books 
WHERE publication_date > '2000-01-01'

'''
num1 = pd.io.sql.read_sql(query1, con=engine)
num1

,count
0,819


There are 821 books published after January 1, 2000.

### 2. Find the number of user reviews and the average rating for each book.

In [3]:
query2='''

 WITH num_of_reviews AS (SELECT book_id, COUNT(DISTINCT review_id) AS num_of_reviews
                                    FROM reviews
                                    GROUP BY book_id),
                                    
                  average_rating AS (SELECT book_id, AVG(rating) AS avg_rating
                                     FROM ratings
                                     GROUP BY book_id)
            SELECT b.book_id, b.title, re.num_of_reviews, ra.avg_rating
            FROM books AS b
            JOIN num_of_reviews AS re ON b.book_id = re.book_id
            JOIN average_rating AS ra ON b.book_id = ra.book_id
            ORDER BY num_of_reviews DESC, avg_rating DESC
            LIMIT 15

'''
num2 = pd.io.sql.read_sql(query2, con=engine)
num2


,book_id,title,num_of_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,656,The Book Thief,6,4.264151
4,734,The Glass Castle,6,4.206897
5,497,Outlander (Outlander #1),6,4.125000
6,750,The Hobbit or There and Back Again,6,4.125000
7,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
8,779,The Lightning Thief (Percy Jackson and the Oly...,6,4.080645
9,963,Water for Elephants,6,3.977273


In this task, it is important to look not only at the average of the ratings, but also at the votes cast. This can distort the first impression of the rating of the books. Therefore, one must find a balanced compromise between the average rating of the book and the number of ratings to be able to conclude the quality.

### 3. Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis)

In [4]:
query3='''

SELECT p.publisher, COUNT(b.book_id)
FROM books b
JOIN publishers p
ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY 2 desc


'''
num3 = pd.io.sql.read_sql(query3, con=engine)
num3

,publisher,count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
329,Turtleback,1
330,Atheneum Books for Young Readers: Richard Jack...,1
331,Penguin Signet,1
332,Victor Gollancz,1


Publishers with a high number of books can give us initial indications of which books are particularly popular at the moment.

### 4. Identify the author with the highest average book rating: look only at books with at least 50 ratings

In [5]:
query4='''


   WITH over_50_ratings AS (SELECT book_id, AVG(rating) 
                                     FROM ratings
                                     GROUP BY book_id
                                     HAVING COUNT(rating_id) > 50)
                                     
            SELECT b.author_id, a.author, AVG(ra.avg) AS avg_book_rating
            FROM books AS b
            JOIN over_50_ratings AS ra ON b.book_id = ra.book_id
            JOIN authors AS a ON a.author_id = b.author_id
            GROUP BY b.author_id, a.author
            ORDER BY avg_book_rating DESC

'''


num4 = pd.io.sql.read_sql(query4, con=engine)
num4

,author_id,author,avg_book_rating
0,236,J.K. Rowling/Mary GrandPré,4.283844
1,402,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,240,J.R.R. Tolkien,4.258446
3,376,Louisa May Alcott,4.192308
4,498,Rick Riordan,4.080645
5,621,William Golding,3.901408
6,235,J.D. Salinger,3.825581
7,469,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,630,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,106,Dan Brown,3.754540


We should put our focus on the authors who have received a high rating and have received at least 50 votes

### 5. Find the average number of text reviews among users who rated more than 50 books

In [6]:



query5 = ''' 
            WITH rated_over_50 AS (SELECT username
                                   FROM ratings
                                   GROUP BY username
                                   HAVING COUNT(rating_id) > 50)
            SELECT AVG(count) AS average_text_reviews
            FROM (SELECT re.username, COUNT(re.review_id)
                  FROM reviews AS re
                  JOIN rated_over_50 AS ra ON ra.username = re.username
                  GROUP BY re.username) AS count
                  '''

num5 = pd.io.sql.read_sql(query5, con=engine)
num5





,average_text_reviews
0,24.333333


Users who have rated a book particularly often also tend to write a short review about a book.

In [7]:
books

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


In [8]:
ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
...,...,...,...,...
6451,6452,1000,carolrodriguez,4
6452,6453,1000,wendy18,4
6453,6454,1000,jarvispaul,5
6454,6455,1000,zross,2


In [9]:
reviews

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
...,...,...,...,...
6451,6452,1000,carolrodriguez,4
6452,6453,1000,wendy18,4
6453,6454,1000,jarvispaul,5
6454,6455,1000,zross,2


In [10]:
authors

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
...,...,...
631,632,William Strunk Jr./E.B. White
632,633,Zadie Smith
633,634,Zilpha Keatley Snyder
634,635,Zora Neale Hurston


In [11]:
DeepnoteChart(books, """{"mark":{"type":"bar","tooltip":true},"$schema":"https://vega.github.io/schema/vega-lite/v4.json","encoding":{"x":{"sort":null,"type":"nominal","field":"","scale":{"type":"linear","zero":false}},"y":{"sort":null,"type":"nominal","field":"","scale":{"type":"linear","zero":true}},"color":{"sort":null,"type":"nominal","field":"","scale":{"type":"linear","zero":false}}},"height":440,"width":"container","autosize":{"type":"fit"},"config":{"legend":{"labelFont":"Inter, sans-serif","labelFontSize":12,"titleFont":"Inter, sans-serif","titleFontSize":12,"titleFontWeight":"bold"},"title":{"anchor":"start","color":"#000000","font":"Inter, sans-serif","fontSize":22,"fontWeight":"normal"},"axis":{"labelFont":"Inter, sans-serif","labelFontSize":12,"labelFontWeight":"normal","titleFont":"Inter, sans-serif","titleFontSize":12,"titleFontWeight":"bold","labelOverlap":"greedy"}}}""")

In [12]:
books

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8cc0f95e-9bd4-4488-91ff-7cfc59de48a4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>